In [ ]:
import keras
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential




In [ ]:
# Load in the Pink Floyd song lyrics data
lyrics = []
with open("pink_floyd_lyrics.txt", "r") as f:
    for line in f:
        lyrics.append(line.strip())



In [ ]:
# Tokenize the lyrics
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lyrics)
sequences = tokenizer.texts_to_sequences(lyrics)



In [ ]:
# Get the vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# Pad the sequences so that they are all the same length
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding="pre")

